In [ ]:
##### A RUN UNIQUEMENT SI LE main.py NE FONCTIONNE PAS ######

import os

# summary

# Dossiers
base_path = os.path.join(os.path.expanduser("~"), "Desktop", "Projet_python", "output")
tp2_path = os.path.join(base_path, "clustering_tp2")
tp4_path = os.path.join(base_path, "prediction_tp4")

summary_path = os.path.join(base_path, "summary")
os.makedirs(summary_path, exist_ok=True)

# Chargement des fichiers
df_fin = pd.read_csv(os.path.join(tp2_path, "financial_profile_labels.csv"))
df_risk = pd.read_csv(os.path.join(tp2_path, "risk_profile_labels.csv"))
df_corr = pd.read_csv(os.path.join(tp2_path, "daily_returns_correlation_labels.csv"))
df_preds = pd.read_csv(os.path.join(tp4_path, "j+1_predictions_models_tp4.csv"))

# Renommage colonnes
for df in [df_fin, df_risk, df_corr]:
    df.rename(columns={df.columns[0]: "Company"}, inplace=True)

# Liste des compagnies à analyser
companies = [
    "Tesla", "Samsung", "Tencent", "Alibaba", "Sony",
    "Adobe", "Johnson & Johnson", "Pfizer", "Louis Vuitton (LVMH)"
]

# Fonction pour trouver les pairs dans le même cluster
def find_cluster_peers(df, company_name, cluster_column="Hierarchical_Cluster"):
    if company_name not in df["Company"].values:
        return []
    cluster_id = df[df["Company"] == company_name][cluster_column].values[0]
    same_cluster = df[df[cluster_column] == cluster_id]["Company"].tolist()
    if company_name in same_cluster:
        same_cluster.remove(company_name)
    return same_cluster

# Compilation des résultats
results = []

for company in companies:
    peers_financial = find_cluster_peers(df_fin, company, "Hierarchical_Cluster")
    peers_risk = find_cluster_peers(df_risk, company, "Hierarchical_Cluster")
    peers_corr = find_cluster_peers(df_corr, company, "Hierarchical_Cluster")

    if company in df_preds["Company"].values:
        row = df_preds[df_preds["Company"] == company].iloc[0]
        pred_xgb = row["XGBoost"]
        pred_rf = row["RandomForest"]
        pred_knn = row["KNN"]
        pred_lr = row["Linear"]
    else:
        last_price = pred_xgb = pred_rf = pred_knn = pred_lr = None

    results.append({
        "Company": company,
        "Peers_Financial": peers_financial,
        "Peers_Risk": peers_risk,
        "Peers_Correlation": peers_corr,
        "Pred_XGBoost": pred_xgb,
        "Pred_RF": pred_rf,
        "Pred_KNN": pred_knn,
        "Pred_Linear": pred_lr
    })

# Création du DataFrame final
df_final = pd.DataFrame(results)

file_path = os.path.join(summary_path, "summary.csv")
df_final.to_csv(file_path, index=False)

display(df_final)

,Company,Peers_Financial,Peers_Risk,Peers_Correlation,Pred_XGBoost,Pred_RF,Pred_KNN,Pred_Linear
0,Tesla,[],"[Alphabet, Samsung, AMD, Qualcomm, Baidu]","[Qualcomm, Sony, Amazon, Alphabet, Apple, Netf...",353.26,350.71,329.06,342.58
1,Samsung,[],"[Alphabet, Tesla, AMD, Qualcomm, Baidu]","[ICBC, Reliance Industries, Tata Consultancy S...",57198.88,57152.93,57786.57,57212.68
2,Tencent,"[Microsoft, Amazon, Alphabet, Meta, IBM, Intel...","[Apple, Microsoft, Amazon, Alibaba, IBM, Intel...","[BYD, Alibaba, Baidu, JD.com]",66.28,65.34,65.13,66.68
3,Alibaba,"[Johnson & Johnson, Pfizer, TotalEnergies, Bai...","[Apple, Microsoft, Amazon, Tencent, IBM, Intel...","[BYD, Baidu, Tencent, JD.com]",122.44,124.16,113.58,125.63
4,Sony,"[Microsoft, Amazon, Alphabet, Meta, Tencent, I...","[Apple, Microsoft, Amazon, Tencent, Alibaba, I...","[Qualcomm, Amazon, Alphabet, Apple, Netflix, A...",24.41,24.56,24.11,24.86
5,Adobe,"[Microsoft, Amazon, Alphabet, Meta, Tencent, I...","[Apple, Microsoft, Amazon, Tencent, Alibaba, I...","[Qualcomm, Sony, Amazon, Alphabet, Apple, Netf...",400.96,405.04,371.42,404.89
6,Johnson & Johnson,"[Alibaba, Pfizer, TotalEnergies, Baidu, JD.com...","[Apple, Microsoft, Amazon, Tencent, Alibaba, I...","[Pfizer, Visa, IBM, JP Morgan, ExxonMobil, Gol...",NaN,NaN,NaN,NaN
7,Pfizer,"[Alibaba, Johnson & Johnson, TotalEnergies, Ba...","[Apple, Microsoft, Amazon, Tencent, Alibaba, I...","[Johnson & Johnson, Visa, IBM, JP Morgan, Exxo...",24.42,24.28,25.95,22.61
8,Louis Vuitton (LVMH),"[Microsoft, Amazon, Alphabet, Meta, Tencent, I...","[Apple, Microsoft, Amazon, Tencent, Alibaba, I...","[TotalEnergies, ASML]",NaN,NaN,NaN,NaN
